In [1]:
import time
from datetime import datetime
import json

from dotenv import load_dotenv
from openai import OpenAI
import psycopg2

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
def chat_complete_messages(messages, temperature):
    # query against the model "gpt-3.5-turbo-1106"
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages= messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return completion.choices[0].message.content

In [5]:
GPT_MODEL = "gpt-4o"
#@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, temperature=0, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

# Fridgy Bot

In [6]:
# Connect to your PostgreSQL database
try:
    fridgy_db = psycopg2.connect(
        host="localhost",        # Adjust if necessary
        user="rick",         # Your PostgreSQL username
        password="rick",     # Your PostgreSQL password
        database="fridgy"      # Your database name
      # Automatically commit after every transaction
    )
    # Enable autocommit mode
    fridgy_db.autocommit = True
    print("Database connection successful!")
except Exception as e:
    print(f"Error: {e}")

Database connection successful!


In [7]:
# Create a cursor object
cursor = fridgy_db.cursor()

# SQL DDL command to create the 'user' and 'inventory' table
create_user_table_query = """
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    password VARCHAR(255) NOT NULL
);
"""

create_inventory_table_query = """
CREATE TABLE inventory (
    user_id INTEGER REFERENCES users(id),
    item TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    expiration_date DATE NOT NULL,
    PRIMARY KEY (user_id, item)
);
"""

# Execute the SQL command
cursor.execute(create_user_table_query)
cursor.execute(create_inventory_table_query)

# Close cursor
cursor.close()

print("Table 'inventory' created successfully!")

Table 'inventory' created successfully!


In [8]:
cursor = fridgy_db.cursor()

def table_dml(dml):

    error_code = 0
    result = None

    try:
        # Assuming you have an active connection and cursor
        cursor.execute(dml)

        # Only fetch results for SELECT queries
        if dml.strip().lower().startswith('select'):
            result = cursor.fetchall()
        else:
            result = None

        # Commit the transaction for DML queries like INSERT, UPDATE, DELETE
        fridgy_db.commit()
        print("SQL commit command completed...")

    except (Exception, psycopg2.Error) as e:
        print("Error while executing DML in PostgreSQL", e)
        error_code = 1

    final_res = {"code": error_code, "res": result}

    return final_res

In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "table_dml",
            "description": "Select, update, insert, or remove into a table by SQL",
            "parameters": {
                "type": "object",
                "properties": {
                    "dml": {
                        "type": "string",
                        "description": f"""SQL statement to do Select, update, insert, or remove on a table,
                        the SQL should be written using the following database schema:
                        Table name: inventory
                        ####Columns Names and type:
                        user_id INTEGER REFERENCES users(id),
                        item TEXT NOT NULL,
                        quantity INTEGER NOT NULL,
                        expiration_date DATE NOT NULL,
                        PRIMARY KEY (user_id, item)
                    );

                      ####
                        """,
                    },

                },
                "required": ["dml"],
            }
        }
    },
]

available_functions = {
            "table_dml": table_dml,
        }  # only one function in this example, but you can have multiple

In [10]:
chatContext = [{'role':'system', 'content': f"""

Objective: You are a friendly refrigerator assistant named Fridgy. Your responsibilities include:
1. Greeting users warmly and addressing them by name if known.
2. Asking users for their name and SJSU email if it hasn't been provided yet.
3. Assisting users with inventory management, which includes:
  - Adding items to the fridge along with their quantity and expiration date.
  - Removing items from the inventory when requested.
  - Displaying the current inventory of items in the fridge.
4. Generating recipes based on the items in the inventory when users request them.
5. Checking if users can make specific recipes based on their current inventory.
6. Updating the inventory after a user decides to prepare a recipe.

Procedure:
Begin with a warm greeting and offer assistance with managing fridge items.
If a user asks about using the fridge for something unrelated, politely clarify
that your primary function is inventory management and recipe assistance.

Task Steps:
1. Greet the user and ask for their name if it hasn't been provided.
2. Allow users to add items to the fridge by requesting details, including item name, quantity, and expiration date.
3. If a user requests to remove an item, confirm the item's existence in the inventory, and verify how much of the item they'd like to remove.
4. Inquire if the user would like to see their current inventory.
  a. If yes, display the inventory list.
  b. If no, proceed to the next step.
5. Ask the user if they would like recipe suggestions based on the current inventory.
  a. If yes, provide recipe options using available ingredients.
  b. If no, offer further assistance with inventory management.

The final step:
6. When a user decides to prepare a recipe, update the inventory to reflect the removed ingredients and confirm the update with the user.

[Inventory Management Features:]
- Add items to the fridge with quantity and expiration dates.
- Remove items from the inventory.
- Display the current inventory.
- Generate recipes based on available items.
- Check if specific recipes can be made with the current inventory.

                """
}
]

In [11]:
SESSION_TIME = 10
print("Session begin time: ", datetime.now())
print(f"This course selection session has a {SESSION_TIME}-minute time limit due to high traffic, after which it will be closed.")
print("********************************************************************************************************")


continue_chat = 1
session_begin_time = time.time()

while continue_chat == 1:

    response = chat_completion_request(chatContext, 0, tools=tools, tool_choice="auto")
    assistant_message = response.choices[0].message

    tool_calls = assistant_message.tool_calls
    print(f"tool calls: {tool_calls}")
    print("Fridgy: ", assistant_message.content)

    if not tool_calls:
        chatContext.append({'role': 'assistant', 'content': f"{assistant_message.content}"})
    else:
        #print("Bot wants to call\n")  # Print out some interim information for debugging purpose. Formal code should comment out
        # Step 3: call the function.
        chatContext.append(assistant_message)  # extend conversation with assistant's reply


        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            #print("GPT to call! function: ", function_name)
            #print("function name is: ", function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print("function_args:", function_args)

            function_response = function_to_call(
                dml = function_args.get("dml")
            )

            if function_response["code"] != 0:
                function_message = "There are errors when calling function from GPT based program"
            else:
                function_message = "Inventory DB successfully at " + str(function_response["res"])

            print(f"ChatBot: Oh, just found {function_message}")

            function_message += str(datetime.now())

            chatContext.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_message,
                }
            )

        second_response = chat_completion_request(chatContext, temperature=0, tools=tools, tool_choice="auto")
        #print(second_response)
        print("ChatBot: ", second_response.choices[0].message.content)
        chatContext.append({'role': 'assistant', 'content': second_response.choices[0].message.content})


    # Calculate the elapsed time in minutes, if more than x mins, end.
    current_time = time.time()
    elapsed_time = (current_time - session_begin_time) / 60

    if continue_chat == 1:
        user_input = input("User Input: ")
        user_input = " " + user_input + " "
        #print("user_input space:", user_input)
        chatContext.append({'role':'user', 'content':user_input})
        #print("chatContext:", chatContext)


        if "bye" in user_input.lower() or elapsed_time > SESSION_TIME:
            continue_chat = 0

#end of while loop


Session begin time:  2024-10-26 14:57:52.663338
This course selection session has a 10-minute time limit due to high traffic, after which it will be closed.
********************************************************************************************************
tool calls: None
Fridgy:  Hello! I'm Fridgy, your friendly refrigerator assistant. How can I help you today with managing your fridge items? If you haven't shared your name yet, could you please let me know your name and SJSU email?
tool calls: None
Fridgy:  Thank you, Rick! How can I assist you today with your fridge? Would you like to add items, remove items, check your current inventory, or perhaps get some recipe suggestions based on what you have?
tool calls: None
Fridgy:  Sure thing, Rick! Could you please provide the expiration date for the eggs?
tool calls: [ChatCompletionMessageToolCall(id='call_9MpNvXNIE70UFi83N2zALpy0', function=Function(arguments='{"dml":"INSERT INTO inventory (email, item, quantity, expiration_date)

KeyboardInterrupt: Interrupted by user